# Week 7: Tree-Based Models - RF, GBM, XGBoost, LightGBM

## 🎯 Learning Objectives

By the end of this week, you will understand:
- **Decision Trees**: The building block
- **Random Forest**: Bagging ensemble
- **Gradient Boosting (GBM)**: Sequential learning
- **XGBoost & LightGBM**: Production-grade implementations
- **Finance Applications**: Non-linear patterns, feature importance

---

## Why Trees in Finance?

- Capture non-linear relationships
- Handle mixed feature types
- Built-in feature importance
- No scaling required
- State-of-the-art for tabular data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
print("✅ Libraries loaded!")
print("📚 Week 7: Tree-Based Models Theory")

---

## Part 1: Decision Trees

### The Algorithm

Recursively split data to minimize impurity:

**For Regression (MSE):**
$$MSE = \frac{1}{n}\sum_{i=1}^{n}(y_i - \bar{y})^2$$

**For Classification (Gini):**
$$Gini = 1 - \sum_{k=1}^{K}p_k^2$$

### 🤔 Simple Explanation

A decision tree is like a game of 20 questions. At each node, it asks a yes/no question about a feature. Based on the answer, it goes left or right until it reaches a leaf with a prediction.

In [ ]:
# Generate non-linear financial data
n = 1000
momentum = np.random.randn(n) * 2
volatility = np.abs(np.random.randn(n))
volume = np.random.exponential(1, n)

# Non-linear relationship: returns depend on regimes
returns = np.where(
    (momentum > 0) & (volatility < 0.5),
    0.002 + 0.001 * momentum,  # Good regime: momentum works
    np.where(
        volatility > 1.5,
        -0.001,  # High vol regime: negative
        0.0001 * volume  # Normal: volume matters
    )
) + np.random.randn(n) * 0.005

X = np.column_stack([momentum, volatility, volume])
feature_names = ['Momentum', 'Volatility', 'Volume']

X_train, X_test, y_train, y_test = train_test_split(X, returns, test_size=0.3, random_state=42)

# Fit decision tree
tree = DecisionTreeRegressor(max_depth=4, random_state=42)
tree.fit(X_train, y_train)

print("Decision Tree Results")
print("="*50)
print(f"Train R²: {tree.score(X_train, y_train):.3f}")
print(f"Test R²:  {tree.score(X_test, y_test):.3f}")
print(f"\nFeature Importance:")
for name, imp in zip(feature_names, tree.feature_importances_):
    print(f"  {name}: {imp:.3f}")

---

## Part 2: Random Forest (Bagging)

### The Idea

Build many trees on bootstrapped samples and average predictions.

$$\hat{y} = \frac{1}{B}\sum_{b=1}^{B}T_b(x)$$

### Key Hyperparameters

- **n_estimators**: Number of trees
- **max_depth**: Tree depth (controls overfitting)
- **max_features**: Features per split (adds randomness)

### 🤔 Simple Explanation

Random Forest = "Wisdom of the crowd"
- Build 100+ different trees
- Each sees different data samples
- Average their predictions
- Result: Lower variance, more robust

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Compare single tree vs forest
tree_single = DecisionTreeRegressor(max_depth=10, random_state=42)
forest = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)

tree_single.fit(X_train, y_train)
forest.fit(X_train, y_train)

print("Single Tree vs Random Forest")
print("="*50)
print(f"{'Model':<20} {'Train R²':<12} {'Test R²':<12}")
print("-"*50)
print(f"{'Single Tree':<20} {tree_single.score(X_train, y_train):<12.3f} {tree_single.score(X_test, y_test):<12.3f}")
print(f"{'Random Forest':<20} {forest.score(X_train, y_train):<12.3f} {forest.score(X_test, y_test):<12.3f}")

print(f"\n✅ Random Forest has better generalization!")

---

## Part 3: Gradient Boosting

### The Idea: Learn from Mistakes

Build trees sequentially, each correcting the errors of the previous:

$$F_m(x) = F_{m-1}(x) + \eta \cdot h_m(x)$$

Where $h_m$ is trained on the **residuals** of $F_{m-1}$.

### Key Hyperparameters

- **learning_rate** (η): Step size (smaller = more trees needed)
- **n_estimators**: Number of boosting stages
- **max_depth**: Usually shallow (3-8)

### 🤔 Simple Explanation

Gradient Boosting is like hiring consultants:
1. First consultant makes predictions
2. Second consultant focuses on what the first got wrong
3. Third consultant fixes what the first two missed
4. Final prediction = sum of all consultants

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Gradient Boosting with different learning rates
learning_rates = [0.01, 0.1, 0.3]

print("Gradient Boosting: Learning Rate Effect")
print("="*50)

for lr in learning_rates:
    gbm = GradientBoostingRegressor(
        n_estimators=100,
        learning_rate=lr,
        max_depth=4,
        random_state=42
    )
    gbm.fit(X_train, y_train)
    print(f"LR={lr}: Train R²={gbm.score(X_train, y_train):.3f}, Test R²={gbm.score(X_test, y_test):.3f}")

---

## Part 4: XGBoost & LightGBM

### XGBoost Innovations

- **Regularization**: L1/L2 on leaf weights
- **Second-order gradients**: Better optimization
- **Missing value handling**: Built-in
- **Parallel learning**: Fast training

### LightGBM Innovations

- **Leaf-wise growth**: Grows deepest leaf first
- **Histogram binning**: Faster splits
- **GOSS**: Gradient-based One-Side Sampling
- **Even faster** than XGBoost

In [ ]:
try:
    import xgboost as xgb
    import lightgbm as lgb
    
    # XGBoost
    xgb_model = xgb.XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=4,
        random_state=42
    )
    xgb_model.fit(X_train, y_train)
    
    # LightGBM
    lgb_model = lgb.LGBMRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=4,
        random_state=42,
        verbose=-1
    )
    lgb_model.fit(X_train, y_train)
    
    print("XGBoost vs LightGBM")
    print("="*50)
    print(f"XGBoost:  Test R² = {xgb_model.score(X_test, y_test):.3f}")
    print(f"LightGBM: Test R² = {lgb_model.score(X_test, y_test):.3f}")
    
except ImportError:
    print("⚠️ XGBoost or LightGBM not installed.")
    print("Install with: pip install xgboost lightgbm")

---

## Part 5: Feature Importance

### Methods

1. **Impurity-based**: How much does splitting on this feature reduce MSE/Gini?
2. **Permutation-based**: How much does performance drop if we shuffle this feature?
3. **SHAP**: Game-theoretic approach (more in Week 11)

In [ ]:
from sklearn.inspection import permutation_importance

# Compare importance methods
impurity_imp = forest.feature_importances_

perm_result = permutation_importance(forest, X_test, y_test, n_repeats=10, random_state=42)
perm_imp = perm_result.importances_mean

print("Feature Importance Comparison")
print("="*50)
print(f"{'Feature':<15} {'Impurity':<15} {'Permutation':<15}")
print("-"*50)
for name, imp, perm in zip(feature_names, impurity_imp, perm_imp):
    print(f"{name:<15} {imp:<15.3f} {perm:<15.3f}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].barh(feature_names, impurity_imp)
axes[0].set_xlabel('Importance')
axes[0].set_title('Impurity-based Importance')

axes[1].barh(feature_names, perm_imp)
axes[1].set_xlabel('Importance')
axes[1].set_title('Permutation Importance')

plt.tight_layout()
plt.show()

---

## Interview Questions

### Conceptual
1. What's the difference between bagging and boosting?
2. Why are Random Forests less prone to overfitting than single trees?
3. How does XGBoost handle missing values?

### Technical
1. Derive the gradient boosting update rule.
2. What is the bias-variance tradeoff for ensembles?
3. How would you tune a gradient boosting model?

### Finance-Specific
1. Why might tree models work well for regime detection?
2. How would you prevent lookahead bias with tree models?
3. When would you prefer linear models over trees?

---

## Key Takeaways

| Model | Ensemble Type | Strengths | Weaknesses |
|-------|--------------|-----------|------------|
| Random Forest | Bagging | Robust, parallel | Can't extrapolate |
| GBM | Boosting | Accurate, handles bias | Slower training |
| XGBoost | Boosting | Regularized, fast | Complex tuning |
| LightGBM | Boosting | Fastest, large data | Memory for large trees |